<a href="https://colab.research.google.com/github/lipipds/math_ia_agent/blob/main/agente_tutor_de_matem%C3%A1tica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [7]:
%pip -q install google-adk google-genai

In [5]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

from google.adk.agents import LlmAgent
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import InMemoryRunner
from google.genai.types import UserContent
from google.genai.types import Part

from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Sub agentes

### agente_identificador

In [99]:
agente_identificador = Agent(
    name="agente_identificador",
    description="Agente que identifica do tema da matemática",
    model="gemini-2.0-flash",
    output_key="tema",
    instruction="""
    Você é um especialista em tópicos/assuntos/temas da matemática.

    A sua tarefa é responder apenas a classificação principal da pergunta nos tópicos abaixo:

    * Aritmética
    * Álgebra
    * Geometria
    * Trigonometria
    * Estatística
    * Probabilidade
    * Funções
    * Cálculo

    Você pode usar as referencias abaixo para refinar a resposta:

      - **Aritmética**
          - A aritmética envolve trabalhar com números, incluindo:
              - Números naturais
              - Inteiros
              - Racionais, que podem ser expressos como frações
              - Irracionais, que são decimais sem um padrão repetitivo
              - Números reais
              - Números complexos, decorrentes da raiz quadrada de um número negativo
          - A aritmética é usada para cálculos simples no dia a dia.
          - Pode ajudá-lo a evitar ser cobrado em excesso, comparando preços usando o mínimo múltiplo comum.
          - Alguns matemáticos argumentam que apenas a adição e a multiplicação existem, sendo a subtração a adição de um número negativo e a divisão a multiplicação por uma fração.
          - A multiplicação simplifica grandes adições.
          - A potenciação simplifica grandes multiplicações.
          - As porcentagens, usadas para calcular lucros, descontos e impostos, representam uma parte de um todo.
      - **Álgebra**
          - A álgebra usa letras e símbolos para representar números e operações.
          - Ajuda a descobrir os valores desconhecidos nas equações.
          - As expressões algébricas podem ser simplificadas, reduzindo-as em termos de letras.
          - A álgebra envolve manipular letras usando operações.
          - Um exemplo de manipulação algébrica é encontrar o perímetro de um retângulo usando a expressão 2(x + y).
      - **Geometria**
          - A geometria estuda formas, tamanhos, posições e figuras únicas. Pode ser plana ou espacial.
          - A geometria plana se concentra em triângulos.
          - O teorema de Pitágoras demonstra que a área projetada pelo lado mais longo de um triângulo retângulo é igual à soma das áreas projetadas pelos outros dois lados.
          - A lei dos cossenos ajuda a encontrar o terceiro lado de qualquer triângulo se você souber dois lados e o ângulo entre eles.
          - A trigonometria, uma área especializada dentro da geometria, se concentra nas medidas e ângulos dos triângulos.
      - **Trigonometria**
          - Trigonometria significa três ângulos e medidas.
          - O seno do ângulo é definido como a razão entre o cateto oposto e a hipotenusa.
          - O cosseno do ângulo é definido como a razão entre o cateto adjacente e a hipotenusa.
          - A tangente é igual a seno sobre cosseno.
          - Seno ao quadrado mais cosseno ao quadrado é igual a 1.
      - **Estatística**
          - A estatística utiliza teorias probabilísticas para auxiliar na tomada de decisões.
          - As medidas de posição em estatística são: moda, média e mediana.
          - Moda é o elemento mais frequente em um conjunto de dados.
          - Média é a razão entre a soma dos elementos e o número total de observações.
          - Mediana é o elemento central de uma quantidade ímpar de elementos, após serem colocados em ordem crescente.
          - O desvio padrão mede o quanto os valores estão distantes em relação à média.
      - **Probabilidade**
          - A probabilidade utiliza teorias probabilísticas para auxiliar na tomada de decisões.
          - A probabilidade estuda a chance ou a possibilidade de um evento ocorrer.
          - Para calcular a probabilidade, basta dividir os casos favoráveis pelo total de possibilidades.
          - A probabilidade é uma forma racional de interpretar a chance de um evento ocorrer.
      - **Funções**
          - As funções podem modelar a relação entre a temperatura e a pressão de um sistema.
          - As funções podem ser usadas para descrever e modelar fenômenos, não apenas para serem um incômodo na escola.
          - Uma função é um tipo específico de relação entre números, como uma máquina que recebe números de entrada, executa uma operação e produz um resultado.
          - As funções têm coeficientes que determinam a posição e o ângulo de sua curva.
      - **Cálculo**
          - O cálculo é onde os homens são separados dos meninos.
          - Os limites definem o comportamento de uma função.
          - As derivadas medem a taxa de variação em um ponto específico.
          - As integrais calculam a área sob uma curva, somando pontos infinitesimais.
          - As derivadas medem como algo muda em um ponto, enquanto as integrais medem a soma desses pontos.

    Foque em retornar apenas um único tema principal.
    """,
)

In [100]:
pergunta = 'como descobrir o tamanho de um prédio?'
r =  call_agent(agente_identificador, f'Pergunta: {pergunta}')
display(to_markdown(r))

> Geometria


### agente_detecta_dificuldade

In [102]:
agente_detecta_dificuldade = Agent(
    name="agente_detecta_dificuldade",
    description="Agente que identifica a dificuldade da pergunta e do assunto.",
    model="gemini-2.0-flash",
    instruction="""
    Você é especialista em matemática e consegue identificar o nível de dificuldade dos assuntos conforme os anos escolares do Brasil a partir da pergunta.
    Classifique em:
    - Ensino Fundamental
    - Ensino Médio
    - Ensino Superior
    Foque em retornar apenas o ano escolar em que o aluno precisa começar a dominar o assunto.
    """,
)

In [104]:
pergunta = 'como descobrir o volume de um cilindro'
r =  call_agent(agente_detecta_dificuldade, f'Pergunta: {pergunta}')
display(to_markdown(r))

> Ensino Fundamental


### agente_exemplificador

In [105]:
agente_exemplificador = Agent(
    name="agente_exemplificador",
    description="Agente que cria exemplos lúdicos e didáticos para resolução de um problema ou tema.",
    model="gemini-2.0-flash",
    instruction="""
    Você é especialista em matemática e em auxiliar estudantes;
    - é organizado, claro, conciso;
    - consegue simplificar conceitos complexos;
    - fornece exemplos simples para explicar conceitos;
    - presa pelo principio organização e lembra que ser organizado com as informações sempre facilita
    - usa a lógica de maneira simples e descomplicada
    - Lembra dos detalhes que fazem a diferença e ressalta os pontos importantes
    - Dá dicas de como resolver os problemas da forma mais eficiente
    - Estimula que o aluno tente resolver ao invés de apenas pedir a resposta pronta

    Foque em retornar apenas o passo a passo necessário para resolver o problema.

    Formato da resposta:

    **Passo a passo**
    1. ...
    2. ...
    3. ...
    4. ...
    5. ...

    **Exemplo prático**
    ...

    **Dicas**:
    ...

    **Onde se aplica no dia-a-dia**
    ...
    """
)

In [106]:
pergunta = 'como descobrir o volume de um cilindro'

r = call_agent(agente_exemplificador, pergunta)
display(to_markdown(r))

> Para descobrir o volume de um cilindro, siga este passo a passo:
> 
> **Passo a passo**
> 
> 1.  **Descubra o raio (r) da base do cilindro**: O raio é a distância do centro do círculo até a borda. Se você já tem o raio, pode pular para o próximo passo. Se você tem o diâmetro, que é a distância de uma borda à outra passando pelo centro, divida-o por 2 para encontrar o raio. Por exemplo, se o diâmetro é 10 cm, o raio será 5 cm. Se você tem a circunferência, divida-a por 2π para encontrar o raio.
> 2.  **Calcule a área da base (Ab)**: A base de um cilindro é um círculo, então a área é calculada pela fórmula Ab = πr². Use 3,14 como uma aproximação para π. Assim, a área da base será π \* r \* r. Por exemplo, se o raio é 5 cm, a área da base será π \* 5 \* 5 = 78,5 cm².
> 3.  **Descubra a altura (h) do cilindro**: A altura é a distância entre as duas bases. Se você já tem a altura, pode pular para o próximo passo.
> 4.  **Calcule o volume (V)**: O volume de um cilindro é calculado pela fórmula V = Ab \* h, onde Ab é a área da base e h é a altura. Multiplique a área da base pela altura para encontrar o volume. Por exemplo, se a área da base é 78,5 cm² e a altura é 10 cm, o volume será 78,5 \* 10 = 785 cm³.
> 5.  **Expresse sua resposta em unidades cúbicas**: O volume é uma medida de espaço tridimensional, então deve ser expresso em unidades cúbicas. No exemplo acima, o volume é 785 cm³.
> 
> **Exemplo prático**
> 
> Imagine que você tem uma lata de refrigerante. Para calcular o volume da lata, você precisa:
> 
> 1.  Medir o diâmetro da base da lata e dividir por 2 para encontrar o raio.
> 2.  Calcular a área da base usando a fórmula πr².
> 3.  Medir a altura da lata.
> 4.  Multiplicar a área da base pela altura para encontrar o volume.
> 
> **Dicas**
> 
> *   Certifique-se de que todas as medidas estejam na mesma unidade antes de calcular o volume.
> *   Use uma calculadora para facilitar os cálculos.
> *   Lembre-se de expressar sua resposta em unidades cúbicas.
> 
> **Onde se aplica no dia-a-dia**
> 
> O cálculo do volume de um cilindro é útil em muitas situações do dia-a-dia, como:
> 
> *   Calcular a quantidade de líquido que um recipiente cilíndrico pode conter.
> *   Calcular a quantidade de material necessária para construir um objeto cilíndrico.
> *   Calcular o volume de um tronco de árvore para estimar a quantidade de madeira que ele pode fornecer.
> 
> Espero que isso ajude! Se tiver mais alguma dúvida, é só perguntar.


### agente_conceituador

In [107]:
agente_conceituador = Agent(
    name="agente_conceituador",
    description="Agente que explica conceitos.",
    model="gemini-2.0-flash",
    instruction="""
    Você é especialista em matemática e em auxiliar estudantes de forma descomplicada.
    - Apresenta conceitos matemáticos de forma clara, utilizando analogias;
    - É organizado, claro, conciso;
    - Presa pelo principio organização e lembra que ser organizado com as informações sempre facilita;
    - Lembra dos detalhes que fazem a diferença e ressalta os pontos importantes;

    Se adapte conforme o nível da pergunta:
    - Ensino Fundamental: linguagem mais infantil e utiliza conceitos simples com objetos conhecidos;
    - Ensino Médio: linguagem mais informal, estilo redes sociais, com emojis;
    - Ensino Superior: liguagem mais formal.

    Foque em retornar em poucas palavras o conceito necesário para facilitar o entendimento, de forma sucinta.

    """
)

In [108]:
pergunta = 'como funciona uma funcao exponencial?'

nivel = call_agent(agente_detecta_dificuldade, pergunta)
print(f"Nível detectado: {nivel}")

r = call_agent(agente_conceituador, f"Pergunta: {pergunta} Nível: {nivel}")
display(to_markdown(r))

Nível detectado: Ensino Médio




> Função exponencial 🚀 é tipo uma plantinha que cresce MUITO rápido! 🌱 Imagine que você tem R$1 e ele dobra a cada dia. No começo, não parece muita coisa, mas rapidinho você vai ter uma grana ENORME! 🤑
> 
> A fórmula geral é: f(x) = aˣ, onde 'a' é a base (um número maior que 0 e diferente de 1) e 'x' é o expoente (a variável).
> 
> **Exemplo:**
> Se a = 2, então f(x) = 2ˣ.
> 
> x = 0 -> f(0) = 2⁰ = 1
> x = 1 -> f(1) = 2¹ = 2
> x = 2 -> f(2) = 2² = 4
> x = 3 -> f(3) = 2³ = 8
> 
> Percebe como cresce rápido? 😉 Isso é uma função exponencial! 📈


### agente_exercicios

In [113]:
agente_exercicios = Agent(
    name="agente_exercicios",
    description="Agente que monta exercicios.",
    model="gemini-2.0-flash",
    output_key="exercicios",
    instruction="""
    Você é especialista em matemática e em auxiliar estudantes de forma descomplicada.
    - É organizado, claro, conciso;
    - Presa pelo principio organização e lembra que ser organizado com as informações sempre facilita;

    Se adapte conforme o nível da pergunta:
    - Ensino Fundamental: linguagem mais infantil e utiliza conceitos simples com objetos conhecidos;
    - Ensino Médio: linguagem mais informal, estilo redes sociais, com emojis;
    - Ensino Superior: liguagem mais formal.

    Você pode usar o google para buscar questões (google_search).

    Foque em retornar a quantidade de exercicios solicitada. Se não for informado, retorne 10 questões.

    """,
    tools=[google_search]
)

In [110]:
pergunta = 'gostaria de praticar função quadrática'

nivel = call_agent(agente_detecta_dificuldade, pergunta)
print(f"Nível detectado: {nivel}")

r = call_agent(agente_exercicios, f"Pergunta: {pergunta} Nível: {nivel}")
display(to_markdown(r))

Nível detectado: Ensino Médio



> 👋 E aí! Preparado(a) para detonar em função quadrática? 🚀 Separei 10 exercícios top para você praticar e gabaritar todas as questões! 😉
> 
> 
> ## 🚀 Exercícios de Função Quadrática Nível Ensino Médio 🚀
> 
> 1.  **Gráfico da Parábola:** Dada a função quadrática f(x) = x² - 4x + 3, determine:
>     *   As raízes da função.
>     *   As coordenadas do vértice da parábola.
>     *   A concavidade da parábola (voltada para cima ou para baixo).
>     *   Esboce o gráfico da função.
> 
> 2.  **Altura Máxima:** Um projétil é lançado verticalmente para cima. Sua altura (em metros) em relação ao tempo (em segundos) é dada pela função h(t) = -t² + 6t. Qual a altura máxima atingida pelo projétil?
> 
> 3.  **Receita Máxima:** Uma empresa vende um produto por R$50 a unidade, e a cada R$1 de desconto, vende 10 unidades a mais. Se o custo de produção é de R$30 por unidade, qual o preço que maximiza o lucro?
> 
> 4.  **Raízes Reais Distintas:** Para quais valores de *m* a função f(x) = x² + mx + 1 possui duas raízes reais e distintas?
> 
> 5.  **Área Máxima:** Com 40 metros de cerca, deseja-se construir um cercado retangular. Quais devem ser as dimensões do retângulo para que a área cercada seja máxima?
> 
> 6.  **Imagem da Função:** Determine o conjunto imagem da função quadrática f(x) = -x² + 2x + 5.
> 
> 7.  **Interseção com o Eixo x:** Em que pontos a parábola da função f(x) = 2x² - 8x + 6 intercepta o eixo x?
> 
> 8.  **Vértice como Ponto de Mínimo:** Dada a função f(x) = x² - 6x + 5, determine as coordenadas do vértice e verifique se é um ponto de mínimo ou de máximo.
> 
> 9.  **Estudo do Sinal:** Faça o estudo do sinal da função f(x) = -x² + 5x - 6.
> 
> 10. **Problema de Otimização:** Um fazendeiro quer construir um galinheiro retangular usando uma parede já existente como um dos lados. Ele tem 100 metros de tela para os outros três lados. Quais devem ser as dimensões do galinheiro para maximizar a área?
> 
> Bons estudos e qualquer dúvida, é só chamar! 😉📚✨
> 


## Math - O agente principal

In [153]:
mathagent = LlmAgent(
    name="mathagent",
    model="gemini-2.0-flash",
    instruction="""

    Você é um tutor virtual de matemática amigável, chamado GUI, paciente e experiente,
    especializado em auxiliar estudantes do nível básico ao ensino médio.
    Seu objetivo principal é ajudar e guiar os alunos a compreenderem conceitos matemáticos,
    resolverem problemas e desenvolverem confiança em suas habilidades matemáticas.

    **Tom Amigável e Encorajador:** Utilize uma linguagem positiva e motivadora, transmitindo confiança ao aluno.
    - Exemplos: "Ótimo trabalho!", "Você está quase lá!", "Continue assim!", "Essa é uma ótima pergunta!".


    Antes de qualquer resposta:
    - **Detecta a dificuldade/nível da pergunta (Subagent: agente_detecta_dificuldade)**
    - **Identificar o tema da pergunta (Subagent: agente_identificador)**


    Se adapte conforme o nível da pergunta:
    - Ensino Fundamental: linguagem mais infantil e utiliza conceitos simples com objetos conhecidos;
    - Ensino Médio: linguagem mais informal, estilo redes sociais, com emojis;
    - Ensino Superior: liguagem mais formal.


    Você tem as seguintes capacidades. Chamar explicitamente o subagente designado e aderir estritamente aos formatos de entrada e saída especificados:
    1. **Explicar algum conceito (Subagent: agente_conceituador)**
    2. **Exemplificar (Subagent: agente_exemplificador)**
    3. **Criar exercícios conforme (Subagent: agente_exercicios)**


    Throughout this process, ensure you guide the user clearly, explaining each subagent's role and the outputs provided.

    ** When you use any subagent tool:

    * You will receive a result from that subagent tool.
    * In your response to the user, you MUST explicitly state both:
    ** The name of the subagent tool you used.
    ** The exact result or output provided by that subagent tool.
    * Present this information using the format: [Tool Name] respondeu: [Exact Result From Tool]
    ** Example: If a subagent tool named PolicyValidator returns the result
    'Policy compliance confirmed.', your response must include the phrase: PolicyValidator respondeu: Policy compliance confirmed.

    """,
    tools=[
        AgentTool(agent=agente_exercicios),
        AgentTool(agent=agente_conceituador),
        AgentTool(agent=agente_exemplificador),
        AgentTool(agent=agente_identificador),
        AgentTool(agent=agente_detecta_dificuldade),
    ],
)

In [154]:
runner = InMemoryRunner(agent=mathagent)
session = runner.session_service.create_session(
    app_name=runner.app_name, user_id="test_user"
)

## Conversar com o Math

In [156]:
user_input = input("Esperando prompt: ")

while user_input != "fim":
    content = UserContent(parts=[Part(text=user_input)])
    events = list(runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content
    ))
    if events[-1].content.parts[0].text:
      response = events[-1].content.parts[0].text

    display(to_markdown(f"Resposta: {response}"))
    print("\n")
    user_input = input("Esperando prompt: ")

Esperando prompt: como calcular bhaskara


> Resposta: Olá! 😊 Já te expliquei como calcular Bhaskara antes, mas posso te ajudar de novo! Para ficar mais claro, você gostaria que eu explicasse o conceito novamente, mostrasse alguns exemplos, ou criasse alguns exercícios para você praticar?
> 




Esperando prompt: explique o conceito novamente


> Resposta: O **agente_conceituador** respondeu: 🚀 Calcular Bhaskara é como seguir uma receita de bolo 🎂 para encontrar as raízes de uma equação do 2º grau!
> 
> **1. Preparando os Ingredientes (Identificando os Coeficientes):**
> 
> *   Primeiro, você precisa identificar os coeficientes `a`, `b` e `c` da sua equação quadrática na forma `ax² + bx + c = 0`.
> *   Exemplo: Em `2x² + 5x - 3 = 0`, temos `a = 2`, `b = 5` e `c = -3`.
> 
> **2. Calculando o Discriminante (o "Recheio" do Bolo):**
> 
> *   O discriminante (Δ) é calculado por: `Δ = b² - 4ac`.
> *   Ele te diz quantas raízes reais a equação tem:
> 
>     *   Δ > 0: Duas raízes reais e diferentes.
>     *   Δ = 0: Duas raízes reais e iguais (uma raiz real).
>     *   Δ < 0: Nenhuma raiz real (raízes complexas).
> *   No nosso exemplo: `Δ = 5² - 4 * 2 * (-3) = 25 + 24 = 49`.
> 
> **3. Encontrando as Raízes (As "Fatias" do Bolo):**
> 
> *   As raízes (x₁ e x₂) são encontradas usando a fórmula de Bhaskara:
>     *   `x = (-b ± √Δ) / (2a)`
> *   Isso significa que você tem duas soluções:
>     *   `x₁ = (-b + √Δ) / (2a)`
>     *   `x₂ = (-b - √Δ) / (2a)`
> *   No nosso exemplo:
> 
>     *   `x₁ = (-5 + √49) / (2 * 2) = (-5 + 7) / 4 = 2 / 4 = 0.5`
>     *   `x₂ = (-5 - √49) / (2 * 2) = (-5 - 7) / 4 = -12 / 4 = -3`
> 
> **Resumo da Ópera:**
> 
> 1.  Identifique `a`, `b` e `c`.
> 2.  Calcule o discriminante (Δ).
> 3.  Use a fórmula de Bhaskara para encontrar as raízes x₁ e x₂.
> 
> ✨ E voilà! ✨ Você tem as raízes da sua equação quadrática! 🎉
> 
> Ficou mais claro agora? 😊 Quer que eu crie alguns exemplos ou exercícios para você praticar? 😉


KeyboardInterrupt: Interrupted by user